# Evaluation

In [1]:
%pip  install --upgrade pip
%pip  install torch
%pip  install farm-haystack[colab,ocr,preprocessing,file-conversion,metrics,inference,faiss-gpu,elasticsearch]
%pip  install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 54.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
#Log system metrics
%pip install wandb
import wandb
wandb.init()

In [3]:
# indices for document_store
doc_index = "doc_index"
label_index = "label_index"

## Documentstore for sparse retrievers

In [4]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [5]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [6]:
import os
from haystack.document_stores import ElasticsearchDocumentStore


from haystack.utils import launch_es

launch_es()

import time

time.sleep(30)

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store_sparse = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index=doc_index,
    label_index=label_index,
    embedding_field="emb",
    embedding_dim=768,
    excluded_meta_data=["emb"],
)

#document_store_sparse.write_documents(dicts)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


## Documentstore for dense retrievers

In [4]:
from haystack.document_stores import FAISSDocumentStore

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


In [8]:
document_store_dense_openai = FAISSDocumentStore(faiss_index_factory_str="Flat",
                                                 index=doc_index,
                                                 similarity="dot_product",
                                                 vector_dim=1536)

In [5]:
document_store_dense_multilingual = FAISSDocumentStore(faiss_index_factory_str="Flat",
                                                       index=doc_index,
                                                       similarity="dot_product",
                                                       vector_dim=512)

In [5]:
document_store_dense_dpr = FAISSDocumentStore(faiss_index_factory_str="Flat",
                                              index=doc_index,
                                              similarity="dot_product")
#The embedding vector size. Default: 768

## Write Documents
(Not Needed for evaluation, since eval_data is added later)

In [ ]:
import json

## Load the data from the JSON file
#with open('faq_SQuAD_test.json', 'r', encoding='utf-8') as f:
#    squad_data = json.load(f)
#
## Extract the "context" field from each paragraph and store it as a document
#documents = [{"content": paragraph["context"]} for data in squad_data["data"] for paragraph in data["paragraphs"]]
#
## Write documents to the document store
#document_store_dense.write_documents(documents)

## Add Preprocessor and Evaluation Data to Documentstore

In [6]:
eval_data = "htwg_faq_QuAD.json"

In [7]:
from haystack.nodes import PreProcessor

preprocessor = PreProcessor(
    split_by="word",
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False,
    language="de"
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
document_store_sparse.add_eval_data(
    filename=eval_data,
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor,
)

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 2931.03docs/s]


In [12]:
document_store_dense_openai.add_eval_data(
    filename=eval_data,
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor,
)

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 1804.00docs/s]
Writing Documents: 10000it [00:00, 48216.27it/s]


In [8]:
document_store_dense_multilingual.add_eval_data(
    filename=eval_data,
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor,
)

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 2548.18docs/s]
Writing Documents: 10000it [00:00, 52616.44it/s]


In [8]:
document_store_dense_dpr.add_eval_data(
    filename=eval_data,
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor,
)

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 3045.97docs/s]
Writing Documents: 10000it [00:00, 44398.26it/s]


In [ ]:
print(document_store_dense_openai.get_label_count(index=label_index))
print(document_store_dense_openai.get_document_count(index=doc_index))

66
22


In [ ]:
#document_store_dense.get_all_documents(index=doc_index)

In [ ]:
#document_store_dense.get_all_documents(index=label_index)

## Retriever 1: BM25

In [13]:
from haystack.nodes import BM25Retriever

bm25_retriever = BM25Retriever(document_store=document_store_sparse)
#bm25_retriever_results = bm25_retriever.eval(top_k=2, label_index=label_index, doc_index=doc_index)
#bm25_retriever_results

## Retriever 2: Embedding - OpenAI

In [14]:
from haystack.nodes import EmbeddingRetriever

openai_api_key = "PUT_API_KEY_HERE"

emb_retriever_openai = EmbeddingRetriever(
   document_store=document_store_dense_openai,
   batch_size=8,
   embedding_model="text-embedding-ada-002",     #Embedding Size needs to match with FAISSDocumentstore: 1536
   api_key=openai_api_key,
   max_seq_len=1536
)


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model text-embedding-ada-002


### Calculate Embedding for Evaluation Data

In [15]:
document_store_dense_openai.update_embeddings(emb_retriever_openai, index=doc_index)

INFO:haystack.document_stores.faiss:Updating embeddings for 70 docs...
Calculating embeddings: 100%|██████████| 9/9 [00:03<00:00,  2.42it/s]
Documents Processed: 10000 docs [00:03, 2658.89 docs/s]


In [ ]:
#emb_retriever_openai_results = emb_retriever_openai.eval(top_k=2, label_index=label_index, doc_index=doc_index)
#emb_retriever_openai_results

## Retriever 3: Embedding with sentence-transformers models

In [9]:
from haystack.nodes import EmbeddingRetriever

emb_retriever_multiling = EmbeddingRetriever(
    document_store=document_store_dense_multilingual,
    embedding_model="sentence-transformers/distiluse-base-multilingual-cased-v1", #Embedding Size needs to match with FAISSDocumentstore: 512
    use_gpu=True,
    batch_size=8,
    #scale_score=False,
)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model sentence-transformers/distiluse-base-multilingual-cased-v1


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Calculate Embedding for Evaluation Data

In [10]:
#document_store_dense_allMini.update_embeddings(emb_retriever_2, index=doc_index)
document_store_dense_multilingual.update_embeddings(emb_retriever_multiling, index=doc_index)

INFO:haystack.document_stores.faiss:Updating embeddings for 70 docs...
Updating Embedding:   0%|          | 0/70 [00:00<?, ? docs/s]

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Documents Processed: 10000 docs [00:02, 4894.97 docs/s]


In [ ]:
#emb_retriever_multiling_results = emb_retriever_multiling.eval(top_k=2, label_index=label_index, doc_index=doc_index)
#emb_retriever_multiling_results

## Retriever 4: DPR - FAISS

In [9]:
from haystack.nodes import DensePassageRetriever

dpr_retriever_german = DensePassageRetriever(
    document_store=document_store_dense_dpr,
    query_embedding_model="deepset/gbert-base-germandpr-question_encoder",   # facebook/dpr-question_encoder-single-nq-base
    passage_embedding_model="deepset/gbert-base-germandpr-ctx_encoder"       # facebook/dpr-ctx_encoder-single-nq-base
)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.modeling.model.language_model:Auto-detected model language: german


### Calculate Embedding for Evaluation Data

In [10]:
document_store_dense_dpr.update_embeddings(dpr_retriever_german, index=doc_index)

INFO:haystack.document_stores.faiss:Updating embeddings for 70 docs...
Create embeddings:  80%|████████  | 64/80 [00:01<00:00, 47.28 Docs/s]
Documents Processed: 10000 docs [00:03, 3150.60 docs/s]


In [ ]:
#dpr_retriever_german_results = dpr_retriever_german.eval(top_k=2, label_index=label_index, doc_index=doc_index)
#dpr_retriever_german_results

# Create Eval labels

In [11]:
from haystack.schema import EvaluationResult, MultiLabel




In [17]:
eval_labels_sparse = document_store_sparse.get_all_labels_aggregated(index=label_index)


In [18]:
eval_labels_openai = document_store_dense_openai.get_all_labels_aggregated(index=label_index)


In [ ]:
eval_labels_openai

In [12]:
eval_labels_multi = document_store_dense_multilingual.get_all_labels_aggregated(index=label_index)


In [12]:
eval_labels_dpr = document_store_dense_dpr.get_all_labels_aggregated(index=label_index)

## Reader 1: Deepset German Model / 335M parameters

In [13]:
#from haystack.pipelines import ExtractiveQAPipeline
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/gelectra-large-germanquad", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/gelectra-large-germanquad' (Electra)


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:Auto-detected model language: german
INFO:haystack.modeling.model.language_model:Loaded 'deepset/gelectra-large-germanquad' (Electra model) from model hub.


tokenizer_config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
#reader_eval_results = reader.eval(document_store=document_store_dense_multilingual, label_index=label_index, doc_index=doc_index)
#reader_eval_results

since isolated reader evaluation gives correct context pieces, type of document_store is irrelevant here.

### Build pipelines

In [14]:
from haystack.pipelines import ExtractiveQAPipeline

In [21]:
pipe_gelectra_bm25 = ExtractiveQAPipeline(reader = reader, retriever = bm25_retriever)

In [22]:
pipe_gelectra_openai = ExtractiveQAPipeline(reader = reader, retriever = emb_retriever_openai)

In [15]:
pipe_gelectra_multiling = ExtractiveQAPipeline(reader = reader, retriever = emb_retriever_multiling)

In [15]:
pipe_gelectra_dpr = ExtractiveQAPipeline(reader = reader, retriever = dpr_retriever_german)

### Isolated Evaluation
measures the best possible results, since correct context piece is provided to reader node

In [23]:
pipe_gelectra_bm25_isolated_spo = pipe_gelectra_bm25.eval(
    labels=eval_labels_sparse, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts",sas_use_gpu=True, add_isolated_node_eval=True
)
pipe_gelectra_bm25_isolated_spo.save("pipe_gelectra_bm25_isolated_spo")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 20.83 Batches/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/479k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO:haystack.schema:Saving evaluation results to pipe_gelectra_bm25_isolated_spo


In [24]:
metrics_isolated = pipe_gelectra_bm25_isolated_spo.calculate_metrics(eval_mode='isolated')
metrics_isolated

{'Retriever': {},
 'Reader': {'exact_match': 0.015873015873015872,
  'f1': 0.3927590607601699,
  'sas': 0.7107437252998352,
  'num_examples_for_eval': 63.0}}

In [25]:
metrics_integrated = pipe_gelectra_bm25_isolated_spo.calculate_metrics(eval_mode='integrated')
metrics_integrated

{'Retriever': {'recall_multi_hit': 1.0,
  'recall_single_hit': 1.0,
  'precision': 0.1999999999999999,
  'map': 0.9761904761904762,
  'mrr': 0.9761904761904762,
  'ndcg': 0.9824252263605456},
 'Reader': {'exact_match': 0.015873015873015872,
  'f1': 0.40894137924626317,
  'sas': 0.7126258015632629,
  'num_examples_for_eval': 63.0}}

### Integrated Evaluation
measures the results, when retriever provides contexts -> performance depends on retrieval correctness

#### Eval with OpenAI Embedding

In [26]:
pipe_gelectra_openai_spo = pipe_gelectra_openai.eval(
    labels=eval_labels_openai, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts",sas_use_gpu=True
)
pipe_gelectra_openai_spo.save("pipe_gelectra_openai_spo")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.54 Batches/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:haystack.schema:Saving evaluation results to pipe_gelectra_openai_spo


In [27]:
metrics_openai = pipe_gelectra_openai_spo.calculate_metrics()
metrics_openai

{'Retriever': {'recall_multi_hit': 1.0,
  'recall_single_hit': 1.0,
  'precision': 0.1999999999999999,
  'map': 0.9735449735449735,
  'mrr': 0.9735449735449735,
  'ndcg': 0.9803469763038558},
 'Reader': {'exact_match': 0.015873015873015872,
  'f1': 0.41259693123114444,
  'sas': 0.7148523330688477,
  'num_examples_for_eval': 63.0}}

#### Eval with sentence-transformers Embedding

In [16]:
pipe_gelectra_multiling_spo = pipe_gelectra_multiling.eval(
    labels=eval_labels_multi, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts",sas_use_gpu=True
)
pipe_gelectra_multiling_spo.save("pipe_gelectra_multiling_spo")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.52 Batches/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.89 Batches/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/479k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO:haystack.schema:Saving evaluation results to pipe_gelectra_multiling_spo


In [17]:
metrics_multiling = pipe_gelectra_multiling_spo.calculate_metrics()
metrics_multiling

{'Retriever': {'recall_multi_hit': 0.8571428571428571,
  'recall_single_hit': 0.8571428571428571,
  'precision': 0.17142857142857137,
  'map': 0.6894179894179894,
  'mrr': 0.6894179894179894,
  'ndcg': 0.7306380363741704},
 'Reader': {'exact_match': 0.015873015873015872,
  'f1': 0.3850290569231127,
  'sas': 0.6994900703430176,
  'num_examples_for_eval': 63.0}}

#### Eval with DPR Embedding

In [16]:
pipe_gelectra_dpr_spo = pipe_gelectra_dpr.eval(
    labels=eval_labels_dpr, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts",sas_use_gpu=True
)
pipe_gelectra_dpr_spo.save("pipe_gelectra_dpr_spo")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.19 Batches/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/479k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO:haystack.schema:Saving evaluation results to pipe_gelectra_dpr_spo


In [17]:
metrics_dpr = pipe_gelectra_dpr_spo.calculate_metrics()
metrics_dpr

{'Retriever': {'recall_multi_hit': 0.6031746031746031,
  'recall_single_hit': 0.6031746031746031,
  'precision': 0.12063492063492066,
  'map': 0.44206349206349205,
  'mrr': 0.44206349206349205,
  'ndcg': 0.48204914874482546},
 'Reader': {'exact_match': 0.015873015873015872,
  'f1': 0.31987626447686596,
  'sas': 0.6675589680671692,
  'num_examples_for_eval': 63.0}}

## Reader 2: Telekom German-English Model / 177M parameters

In [18]:
from haystack.pipelines import ExtractiveQAPipeline
from haystack.nodes import FARMReader

reader2 = FARMReader(model_name_or_path="deutsche-telekom/bert-multi-english-german-squad2", use_gpu=True)
#extract_pipe = ExtractiveQAPipeline(reader=reader , retriever=retriever)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deutsche-telekom/bert-multi-english-german-squad2' (Bert)


model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:Auto-detected model language: german
INFO:haystack.modeling.model.language_model:Loaded 'deutsche-telekom/bert-multi-english-german-squad2' (Bert model) from model hub.


tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


### Build pipelines

In [19]:
from haystack.pipelines import ExtractiveQAPipeline

In [30]:
pipe_telekom_bm25 = ExtractiveQAPipeline(reader = reader2, retriever = bm25_retriever)

In [31]:
pipe_telekom_openai = ExtractiveQAPipeline(reader = reader2, retriever = emb_retriever_openai)

In [20]:
pipe_telekom_multiling = ExtractiveQAPipeline(reader = reader2, retriever = emb_retriever_multiling)

In [20]:
pipe_telekom_dpr = ExtractiveQAPipeline(reader = reader2, retriever = dpr_retriever_german)

### Isolated Evaluation
measures the best possible results, since correct context piece is provided to reader node

In [32]:
pipe_telekom_bm25_isolated_spo = pipe_telekom_bm25.eval(
    labels=eval_labels_sparse, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts",sas_use_gpu=True, add_isolated_node_eval=True
)
pipe_telekom_bm25_isolated_spo.save("pipe_telekom_bm25_isolated_spo")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 50.10 Batches/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:haystack.schema:Saving evaluation results to pipe_telekom_bm25_isolated_spo


In [33]:
metrics_isolated = pipe_telekom_bm25_isolated_spo.calculate_metrics(eval_mode='isolated')
metrics_isolated

{'Retriever': {},
 'Reader': {'exact_match': 0.0,
  'f1': 0.1928264819195505,
  'sas': 0.6431787610054016,
  'num_examples_for_eval': 63.0}}

In [34]:
metrics_integrated = pipe_telekom_bm25_isolated_spo.calculate_metrics(eval_mode='integrated')
metrics_integrated

{'Retriever': {'recall_multi_hit': 1.0,
  'recall_single_hit': 1.0,
  'precision': 0.1999999999999999,
  'map': 0.9761904761904762,
  'mrr': 0.9761904761904762,
  'ndcg': 0.9824252263605456},
 'Reader': {'exact_match': 0.0,
  'f1': 0.21997046373900941,
  'sas': 0.6510328650474548,
  'num_examples_for_eval': 63.0}}

### Integrated Evaluation
measures the results, when retriever provides contexts -> performance depends on retrieval correctness

#### Eval with OpenAI Embedding

In [35]:
pipe_telekom_openai_spo = pipe_telekom_openai.eval(
    labels=eval_labels_openai, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts",sas_use_gpu=True
)
pipe_telekom_openai_spo.save("pipe_telekom_openai_spo")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 10.11 Batches/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:haystack.schema:Saving evaluation results to pipe_telekom_openai_spo


In [36]:
metrics_openai = pipe_telekom_openai_spo.calculate_metrics()
metrics_openai

{'Retriever': {'recall_multi_hit': 1.0,
  'recall_single_hit': 1.0,
  'precision': 0.1999999999999999,
  'map': 0.9735449735449735,
  'mrr': 0.9735449735449735,
  'ndcg': 0.9803469763038558},
 'Reader': {'exact_match': 0.0,
  'f1': 0.22992581015272573,
  'sas': 0.6542592644691467,
  'num_examples_for_eval': 63.0}}

#### Eval with sentence-transformers Embedding

In [21]:
pipe_telekom_multiling_spo = pipe_telekom_multiling.eval(
    labels=eval_labels_multi, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts",sas_use_gpu=True
)
pipe_telekom_multiling_spo.save("pipe_telekom_multiling_spoc")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  6.67 Batches/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  7.14 Batches/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  9.51 Batches/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 13.51 Batches/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 16.94 Batches/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:haystack.schema:Saving evaluation results to pipe_telekom_multiling_spoc


In [22]:
metrics_multiling = pipe_telekom_multiling_spo.calculate_metrics()
metrics_multiling

{'Retriever': {'recall_multi_hit': 0.8571428571428571,
  'recall_single_hit': 0.8571428571428571,
  'precision': 0.17142857142857137,
  'map': 0.6894179894179894,
  'mrr': 0.6894179894179894,
  'ndcg': 0.7306380363741704},
 'Reader': {'exact_match': 0.0,
  'f1': 0.1999092572905283,
  'sas': 0.6401894688606262,
  'num_examples_for_eval': 63.0}}

#### Eval with DPR Embedding

In [21]:
pipe_telekom_dpr_spo = pipe_telekom_dpr.eval(
    labels=eval_labels_dpr, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts",sas_use_gpu=True
)
pipe_telekom_dpr_spo.save("pipe_telekom_dpr_spo")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  9.28 Batches/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:haystack.schema:Saving evaluation results to pipe_telekom_dpr_spo


In [22]:
metrics_dpr = pipe_telekom_dpr_spo.calculate_metrics()
metrics_dpr

{'Retriever': {'recall_multi_hit': 0.6031746031746031,
  'recall_single_hit': 0.6031746031746031,
  'precision': 0.12063492063492066,
  'map': 0.44206349206349205,
  'mrr': 0.44206349206349205,
  'ndcg': 0.48204914874482546},
 'Reader': {'exact_match': 0.0,
  'f1': 0.17978284148634666,
  'sas': 0.6212383508682251,
  'num_examples_for_eval': 63.0}}

## Reader 3: GPT

In [23]:
from haystack import Pipeline
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser, PromptModel
openai_api_key = "PUT_API_KEY_HERE"

rag_prompt = PromptTemplate(
    prompt="""Synthesize a comprehensive answer from the following text for the given question.
                             Provide a clear and concise response in german that summarizes the key points and information presented in the text.
                             Your answer should be using the given words and be no longer than 50 words.
                             \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    output_parser=AnswerParser(),
)

prompt_node = PromptNode(model_name_or_path="gpt-3.5-turbo-1106", api_key=openai_api_key, default_prompt_template=rag_prompt)

#gptPipeline = Pipeline()
#gptPipeline.add_node(component=retriever, name="BM25Retriever", inputs=["Query"])
#gptPipeline.add_node(component=prompt_node, name="prompt_node", inputs=["BM25Retriever"])

In [ ]:
#eval_labels = document_store_.get_all_labels_aggregated(open_domain=True, drop_negative_labels=False, drop_no_answers=False)

#eval_result_gpt = gptPipeline.eval(labels=eval_labels)

### Build pipelines

In [38]:
pipe_gpt_bm25 = Pipeline()
pipe_gpt_bm25.add_node(component=bm25_retriever, name="Retriever", inputs=["Query"])
pipe_gpt_bm25.add_node(component=prompt_node, name="Reader", inputs=["Retriever"])

In [39]:
pipe_gpt_openai = Pipeline()
pipe_gpt_openai.add_node(component=emb_retriever_openai, name="Retriever", inputs=["Query"])
pipe_gpt_openai.add_node(component=prompt_node, name="Reader", inputs=["Retriever"])

In [24]:
pipe_gpt_multiling = Pipeline()
pipe_gpt_multiling.add_node(component=emb_retriever_multiling, name="Retriever", inputs=["Query"])
pipe_gpt_multiling.add_node(component=prompt_node, name="Reader", inputs=["Retriever"])

In [24]:
pipe_gpt_dpr = Pipeline()
pipe_gpt_dpr.add_node(component=dpr_retriever_german, name="Retriever", inputs=["Query"])
pipe_gpt_dpr.add_node(component=prompt_node, name="Reader", inputs=["Retriever"])

### Isolated Evaluation
measures the best possible results, since correct context piece is provided to reader node

In [40]:
pipe_gpt_bm25_isolated_spo = pipe_gpt_bm25.eval(
    labels=eval_labels_sparse, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts",sas_use_gpu=True, add_isolated_node_eval=True
)
pipe_gpt_bm25_isolated_spo.save("pipe_gpt_bm25_isolated_spo")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:haystack.schema:Saving evaluation results to pipe_gpt_bm25_isolated_spo


In [ ]:
metrics_isolated = pipe_gpt_bm25_isolated_spo.calculate_metrics(eval_mode='isolated')
metrics_isolated

{'Retriever': {}, 'Reader': {}}

In [41]:
metrics_integrated = pipe_gpt_bm25_isolated_spo.calculate_metrics(eval_mode='integrated')
metrics_integrated

{'Retriever': {'recall_multi_hit': 1.0,
  'recall_single_hit': 1.0,
  'precision': 0.1999999999999999,
  'map': 0.9761904761904762,
  'mrr': 0.9761904761904762,
  'ndcg': 0.9824252263605456},
 'Reader': {'exact_match': 0.0,
  'f1': 0.36503612012505704,
  'sas': 0.7207269072532654,
  'num_examples_for_eval': 63.0}}

### Integrated Evaluation
measures the results, when retriever provides contexts -> performance depends on retrieval correctness

#### Eval with OpenAI Embedding

In [42]:
pipe_gpt_openai_spo = pipe_gpt_openai.eval(
    labels=eval_labels_openai, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts",sas_use_gpu=True
)
pipe_gpt_openai_spo.save("pipe_gpt_openai_spo")

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  3.80it/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:haystack.schema:Saving evaluation results to pipe_gpt_openai_spo


In [43]:
metrics_openai = pipe_gpt_openai_spo.calculate_metrics()
metrics_openai

{'Retriever': {'recall_multi_hit': 1.0,
  'recall_single_hit': 1.0,
  'precision': 0.1999999999999999,
  'map': 0.9735449735449735,
  'mrr': 0.9735449735449735,
  'ndcg': 0.9803469763038558},
 'Reader': {'exact_match': 0.0,
  'f1': 0.3572332237629609,
  'sas': 0.721752405166626,
  'num_examples_for_eval': 63.0}}

#### Eval with sentence-transformers Embedding

In [25]:
pipe_gpt_multiling_spo = pipe_gpt_multiling.eval(
    labels=eval_labels_multi, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts",sas_use_gpu=True
)
pipe_gpt_multiling_spo.save("pipe_gpt_multiling_spo")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:haystack.schema:Saving evaluation results to pipe_gpt_multiling_spo


In [26]:
metrics_multiling = pipe_gpt_multiling_spo.calculate_metrics()
metrics_multiling

{'Retriever': {'recall_multi_hit': 0.8571428571428571,
  'recall_single_hit': 0.8571428571428571,
  'precision': 0.17142857142857137,
  'map': 0.6894179894179894,
  'mrr': 0.6894179894179894,
  'ndcg': 0.7306380363741704},
 'Reader': {'exact_match': 0.0,
  'f1': 0.3440141379499412,
  'sas': 0.716810405254364,
  'num_examples_for_eval': 63.0}}

#### Eval with DPR Embedding

In [25]:
pipe_gpt_dpr_spo = pipe_gpt_dpr.eval(
    labels=eval_labels_dpr, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts",sas_use_gpu=True
)
pipe_gpt_dpr_spo.save("pipe_gpt_dpr_spo")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:haystack.schema:Saving evaluation results to pipe_gpt_dpr_spo


In [26]:
metrics_dpr = pipe_gpt_dpr_spo.calculate_metrics()
metrics_dpr

{'Retriever': {'recall_multi_hit': 0.6031746031746031,
  'recall_single_hit': 0.6031746031746031,
  'precision': 0.12063492063492066,
  'map': 0.44206349206349205,
  'mrr': 0.44206349206349205,
  'ndcg': 0.48204914874482546},
 'Reader': {'exact_match': 0.0,
  'f1': 0.323787612175441,
  'sas': 0.6868749856948853,
  'num_examples_for_eval': 63.0}}